## Combining DataFrames for Joining: SkillShed Analysis

### Author: Nghi Nguyen

### Agenda


1. File and Directory Navigation Commands
2. Reading DataFrames from Python Script
3. Performing Joins on DataFrames

In [3]:
from os import getcwd, listdir, chdir

### 1. File and Directory Navigation Commands

- ``getcwd()``

This command is used to get the current working directory. Equivalent of using the `pwd` command in a Terminal or Command Line Interface (CLI).

- ``chdir()``

This command is used to change the current working directory by navigating to the directory mentioned in the input to the function. Equivalent of ``cd`` in Terminal or Command Line Interface (CLI).

- ``listdir()``

This command is used to list the contents of the directory specified as input to the function. Equivalent of ``ls`` on MacOS and ``dir`` on Windows.

In [4]:
chdir("..")

In [5]:
getcwd()

'/Volumes/SGD3/DS/Python/Pathrise/Fellows/Nghi/Pathrise_Nghi_Coding_Tutorials/src'

In [6]:
listdir(getcwd())

['event_driven_programming', 'nb', 'sas_to_pandas']

In [7]:
chdir("sas_to_pandas")

In [9]:
getcwd()

'/Volumes/SGD3/DS/Python/Pathrise/Fellows/Nghi/Pathrise_Nghi_Coding_Tutorials/src/sas_to_pandas'

In [10]:
current = getcwd()
listdir(current)

['4_df_join.py',
 'cleaned_data_euclidean_distance.py',
 'cleaned_data_join.py',
 'cleaned_data_occupations_of_interest.py',
 'cleaned_data_wage_table.py',
 'cleaning_raw_data.py',
 'col_remover.py',
 'dummy_data.py',
 'euclidean_distance.py',
 'knowledge_wide_exploration.py',
 'occupations_of_interest.py',
 'pandas_join_example.py',
 'pandas_transform.py',
 'scratchpad',
 'wage_table.py',
 '__pycache__']

### 2. Reading DataFrames from Python Script

In this section, we'll change the current working directory to be the parent directory of the GitHub repository in order to see the data and the Python scripts for rendering pandas DataFrames for analysis.

In [12]:
chdir("../..")

In [16]:
data_path = getcwd() + "/data/sas_to_pandas/raw/wages"

In [17]:
listdir(data_path)

['OH_OEWS_21.csv', 'raw', 'Wages_Table_Final.csv']

In [21]:
from src.sas_to_pandas.cleaning_raw_data import knowledge_wide, work_activity_wide, work_context_wide, job_zones_2

### 3. Performing Joins on DataFrames

In this section, we utilize the above DataFrames and join them together into a single DataFrame.

In [23]:
## Merge Function
def merge_frames(frames: list, cols_to_join=['SOC_Code']) -> type(knowledge_wide):
    # Join the DataFrames
    result = frames[0]
    for i in range(1, len(frames)):
        result = result.merge(frames[i], on=['SOC_Code'], how='inner')

    return result


# Duplicate Filter Function
def find_duplicate_cols(df):
    # Find Duplicate Columns
    duplicate_cols = []

    for col in df.columns.tolist():
        if '_x' in col:
            duplicate_cols.append(col)
        elif '__x' in col:
            duplicate_cols.append(col)
        elif '_y' in col:
            duplicate_cols.append(col)
        elif '__y' in col:
            duplicate_cols.append(col)
        else:
            continue
    return duplicate_cols

# Keep the Title column
def create_col(df, col: str):

    # Find all columns that contain your input col string
    cols = [elem for elem in df.columns.tolist() if col in elem]

    # Secondary DataFrame
    secondary_df = df[cols]

    # Rename Columns
    secondary_df.columns = [f"col_{i+1}" for i in range(secondary_df.shape[1])]

    # Generate a Series of Cardinality Values for the Above Columns
    cardinality_series = secondary_df.nunique()

    # Choose the record that has the maximum value for cardinality
    cardinality_record = cardinality_series[cardinality_series == max(cardinality_series.values)]

    # Select the col corresponding to the above record
    selected_col = cardinality_record.index[0]

    # Save the Series from the above col
    saved_series = secondary_df[selected_col]

    # Remove the cols
    df.drop(cols, axis=1, inplace=True)

    # Create the new column
    df[col] = saved_series.tolist()

    return df

In [24]:
# Setting Frame Order
## Joins
frame_order = [knowledge_wide, work_activity_wide, work_context_wide, job_zones_2]

# Applying the Functions
result_df = create_col(merge_frames(frame_order), 'Title')

## Reorder the Columns
result_cols = ['SOC_Code', 'Title'] + sorted(result_df.columns.tolist()[1:-1])
result_df = result_df[result_cols]

In [25]:
result_df.shape

(873, 134)